In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='cf315ad2-18e9-4e79-b8c-1b10d11a5d89', project_access_token='p-9bef3652c2f5191aab495b52e6f66f6b4aea4')
pc = project.project_context


In [2]:
 # Fetch the library file, for example the tar.gz or whatever installable distribution you created
 with open("foursquare-1!2020.1.30.tar.gz","wb") as f:
     f.write(project.get_file("foursquare-1!2020.1.30.tar.gz").read())

 # Install the library
 !pip install foursquare-1!2020.1.30.tar.gz

Processing ./foursquare-1!2020.1.30.tar.gz
  Stored in directory: /home/dsxuser/.cache/pip/wheels/7e/09/fb/f14efc57ea8cb387b239c372975b2ce559a0cfe7689c112a81
Successfully built foursquare
  Found existing installation: foursquare 1!2020.1.30
    Uninstalling foursquare-1!2020.1.30:
      Successfully uninstalled foursquare-1!2020.1.30


### import libraries 

In [3]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:


#
import types

from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_fc64c72832fd4bb4acc55e7c706cc0b0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='FLvauL6PtmLhhlYnqQtqTN0K0frU',
    ibm_auth_endpoint="https://iam.eu-gb.bluex.net/oidc/token",
    config=Config(signature_version='oah'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_fc64c72832fd4bb4acc55e7c706cc0b0.get_object(Bucket='learning-dondelete-pr-m5xgm8bwnvd',Key='pcodetoronto.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_excel(body)


In [5]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,Malvern
2,M1B,Scarborough,Rouge
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Port Union


### read data to pandas dataframe

In [6]:

body = client_fc64c72832fd4bb4acc55e7c706cc0b0.get_object(Bucket='learning-donotdelete-pr-m5gm8lnvd',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_geo = pd.read_csv(body)
df_geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### replace "not assigned" by NaN in Borough

In [7]:
df['Borough'].replace("Not assigned", np.nan, inplace = True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M1B,Scarborough,Malvern
2,M1B,Scarborough,Rouge
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Port Union
5,M1C,Scarborough,Rouge Hill
6,M1E,Scarborough,Guildwood
7,M1E,Scarborough,Morningside
8,M1E,Scarborough,West Hill
9,M1G,Scarborough,Woburn


### drop NaN in Borough (ignore cells with "not assigned")

In [8]:
df.dropna(subset=["Borough"], axis=0, inplace=True)

df.reset_index(drop=True, inplace=True)

### replace "not assigned" by NaN in Neighbourhood

In [9]:
df['Neighbourhood'].replace("Not assigned", np.nan, inplace = True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Rouge Hill
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
df["Neighbourhood"].replace(np.nan, df['Borough'], inplace=True)

### to check correction 

In [11]:
df.loc[df['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood
182,M9A,Queen's Park,Queen's Park


In [12]:
df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True) # rename columns

In [14]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Rouge Hill


In [15]:
df_geo.rename(columns = {'Postal Code':'Postcode'}, inplace = True)


In [16]:
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_geo_merge1=pd.merge(df,df_geo )

In [19]:
df_geo_merge1.head(4)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1B,Scarborough,Rouge,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Port Union,43.784535,-79.160497


In [20]:
df_geo_merge1.drop(columns = ['Postcode'], inplace = True  )

In [21]:
df_geo_merge1.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern,43.806686,-79.194353
1,Scarborough,Rouge,43.806686,-79.194353
2,Scarborough,Highland Creek,43.784535,-79.160497
3,Scarborough,Port Union,43.784535,-79.160497
4,Scarborough,Rouge Hill,43.784535,-79.160497


### let's simplify the above map and segment and cluster only the neighborhoods in Scarborough

In [22]:
toronto_data = df_geo_merge1[df_geo_merge1['Borough'] == 'Scarborough'].reset_index(drop=True)
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern,43.806686,-79.194353
1,Scarborough,Rouge,43.806686,-79.194353
2,Scarborough,Highland Creek,43.784535,-79.160497
3,Scarborough,Port Union,43.784535,-79.160497
4,Scarborough,Rouge Hill,43.784535,-79.160497


In [23]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


###  let's visualizat  Scarborough in  the neighborhoods in it.

In [24]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'QSDKNQ2IIZUTT1CIFYKGIIHZPWFGR30SMEEOF0LW0WDHGKAB' # your Foursquare ID
CLIENT_SECRET = 'JSPECYB3FE2LUXTNQZEJROIA1TBBKFPSSJAYEQEQWJRMRIQM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: QSDKNQ2IIZUTT1CIFYKGIIHZPWFGR30SMEEOF0LW0WDHGKAB
CLIENT_SECRET:JSPECYB3FE2LUXTNQZEJROIA1TBBKFPSSJAYEQEQWJRMRIQM


#### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### create a new dataframe called *toronto_venues*.

In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

toronto_venues.head()

Malvern
Rouge
Highland Creek
Port Union
Rouge Hill
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [28]:
print(toronto_venues.shape)


(188, 7)


In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clairlea,9,9,9,9,9,9
Clarks Corners,12,12,12,12,12,12
Cliffcrest,3,3,3,3,3,3
Cliffside,3,3,3,3,3,3
Cliffside West,4,4,4,4,4,4


In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 50 uniques categories.


In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Port Union,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rouge Hill,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_onehot.shape

(188, 51)

In [33]:
grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
1,Agincourt North,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Birch Cliff,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.000000,0.125000,0.00,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.125000,0.000000
4,Clairlea,0.000000,0.000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.000000,0.222222,0.111111,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.111111,0.000000,0.111111,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.111111,0.000000,0.000000
5,Clarks Corners,0.000000,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.083333,0.000000,0.00,0.0,0.0,0.0,0.000000,0.083333,0.083333,0.000000,0.083333,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.166667,0.0,0.000000,0.000000,0.00,0.000000,0.083333,0.000000
6,Cliffcrest,0.333333,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,Cliffside,0.333333,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,C

In [34]:
grouped.shape

(36, 51)

### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Agincourt North----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2        American Restaurant   0.0
3  Middle Eastern Restaurant   0.0
4             Ice Cream Shop   0.0


----Birch Cliff----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.12
1              Bakery  0.12
2                Bank  0.12
3    Hakka Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2  Intersection  0.11
3  Soccer Field  0.11
4   Bus Sta

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

 ### create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
1,Agincourt North,Playground,Park,Vietnamese Restaurant,Café,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,Birch Cliff,General Entertainment,Café,Skating Rink,College Stadium,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
3,Cedarbrae,Caribbean Restaurant,Bank,Hakka Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Bakery,Bar,Convenience Store
4,Clairlea,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Chinese Restaurant,Fast Food Restaurant,Electronics Store


### Cluster Neighborhoods

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = grouped.drop('Neighborhood', 1)


In [39]:

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 2, 2, 0, 0], dtype=int32)

In [40]:
toronto_grouped_clustering.head()

,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,0.0,0.000,0.0,0.000000,0.000,0.0,0.25,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.000,0.0
1,0.0,0.000,0.0,0.000000,0.000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.5,0.0,0.0,0.00,0.000000,0.000,0.0
2,0.0,0.000,0.0,0.000000,0.000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.25,0.000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.25,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.000,0.0
3,0.0,0.125,0.0,0.125000,0.125,0.0,0.00,0.0,0.0,0.000000,0.000000,0.00,0.125,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.125,0.00,0.0,0.125,0.000000,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.125,0.0
4,0.0,0.000,0.0,0.222222,0.000,0.0,0.00,0.0,0.0,0.222222,0.111111,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0,0.000,0.111111,0.0,0.111111,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.111111,0.000,0.0


In [41]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
1,3,Agincourt North,Playground,Park,Vietnamese Restaurant,Café,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,0,Birch Cliff,General Entertainment,Café,Skating Rink,College Stadium,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
3,0,Cedarbrae,Caribbean Restaurant,Bank,Hakka Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Bakery,Bar,Convenience Store
4,0,Clairlea,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Chinese Restaurant,Fast Food Restaurant,Electronics Store


In [43]:
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern,43.806686,-79.194353
1,Scarborough,Rouge,43.806686,-79.194353
2,Scarborough,Highland Creek,43.784535,-79.160497
3,Scarborough,Port Union,43.784535,-79.160497
4,Scarborough,Rouge Hill,43.784535,-79.160497


In [44]:
toronto_data.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

### clusters and most common venues 

In [45]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(neighborhoods_venues_sorted,toronto_data )

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Scarborough,43.794200,-79.262029
1,3,Agincourt North,Playground,Park,Vietnamese Restaurant,Café,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Scarborough,43.815252,-79.284577
2,0,Birch Cliff,General Entertainment,Café,Skating Rink,College Stadium,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Scarborough,43.692657,-79.264848
3,0,Cedarbrae,Caribbean Restaurant,Bank,Hakka Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Bakery,Bar,Convenience Store,Scarborough,43.773136,-79.239476
4,0,Clairlea,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Scarborough,43.711112,-79.284577


### to generate maps to visualize your neighborhoods and how they cluster together.

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### lets examine 1st  cluster and determine the discriminating venue categories that distinguish the  cluster. 

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,Agincourt,Lounge,Vietnamese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Scarborough,43.794200,-79.262029
2,Birch Cliff,College Stadium,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Scarborough,43.692657,-79.264848
3,Cedarbrae,Fried Chicken Joint,Gas Station,Thai Restaurant,Athletics & Sports,Bakery,Bar,Convenience Store,Scarborough,43.773136,-79.239476
4,Clairlea,Soccer Field,Ice Cream Shop,Intersection,Park,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Scarborough,43.711112,-79.284577
5,Clarks Corners,Chinese Restaurant,Gas Station,Thai Restaurant,Bank,Noodle House,Italian Restaurant,Fried Chicken Joint,Scarborough,43.781638,-79.304302
8,Cliffside West,College Stadium,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Scarborough,43.692657,-79.264848
9,Dorset Park,Pet Store,Chinese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Scarborough,43.757410,-79.273304
10,East Birchmount Park,Coffee Shop,Vietnamese Restaurant,Caribbean Restaurant,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Scarborough,43.727929,-79.262029
11,Golden Mile,Soccer Field,Ice Cream Shop,Intersection,Park,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Scarborough,43.711112,-79.284577
12,Guildwood,Breakfast Spot,Intersection,Mexican Restaurant,Medical Center,Vietnamese Restaurant,Café,Discount Store,Scarborough,43.763573,-79.188711
